In this notebook i'll use the sklearn pipeline for the ML project.

The data i'm using was cleaned using this notebook : https://github.com/RaphaelRoriz/Machine_learning/blob/master/housesPricesDataset/House%20Prices%20regressions.ipynb


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
%matplotlib inline

In [19]:
houses_train = pd.read_pickle('houses_train_cleaned.pkl')
houses_train.drop('Id',axis =1,inplace =True )

In [21]:
columns_with_null_values = houses_train.columns[houses_train.isna().any()].tolist()

#getting the name and total number of null values per column
columns_with_null_values_sum = {}
for column_name in columns_with_null_values:
    columns_with_null_values_sum[column_name] = houses_train[column_name].isnull().sum()
    
columns_with_null_values_sum

{}

In [22]:
y = houses_train['SalePrice']
X = houses_train.drop(['SalePrice'],axis = 1)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [24]:
pipe = Pipeline(steps=[
                       ("pca", PCA()),
                       ("regression",RandomForestRegressor(n_estimators=100))])

In [25]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('regression', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_...tors=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [26]:
metrics = ["explained_variance",
            "neg_mean_squared_error",
            "r2"]



In [ ]:
cross_validation_scores = cross_validate(pipe, X, y, 
               return_train_score=False,
               scoring=metrics,cv=5)